In [1]:
import math
import numpy as np
from keras.layers import (
    Dense,
    Activation,
    Dropout,
    Flatten,
    AveragePooling2D,
)
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import LearningRateScheduler
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from google.colab import drive
drive.mount("/content/drive/")

orig_path = '/content/drive/My Drive/bird_classification/' + '/data/bird_dataset/'

Mounted at /content/drive/


In [2]:
# Seed value (can actually be different for each attribution step)
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value) # tensorflow 2.x
# tf.set_random_seed(seed_value) # tensorflow 1.x

# Original Images

## Loading Data

In [3]:
img_rows, img_cols = (334,334)
train_batchsize = 16
val_batchsize = 16

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        orig_path + 'train_images/',
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical',
        interpolation='bicubic')
 
validation_generator = validation_datagen.flow_from_directory(
        orig_path + 'val_images/',
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        interpolation='bicubic')

Found 1082 images belonging to 20 classes.
Found 103 images belonging to 20 classes.


## Utils

In [6]:
# Swish Activation Function
def swish(x):
    return K.sigmoid(x) * x

get_custom_objects().update({"swish": Activation(swish)})


# Learning Step Decay by 10e-1 after every 4 epochs
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch) / epochs_drop))
    return lrate

# Calculates Precision Accuracy
def precision(y_true, y_pred):
    """Precision metric.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# Calculates Recall Accuracy
def recall(y_true, y_pred):
    """Recall metric.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# Calculates F1 score
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

## Phase 1 : Trained only on top layer / Inception No Trainable

In [ ]:
base_model = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(334,334,3)
    )

219062272/219055592 [==============================] - 2s 0us/step


In [ ]:
# Freeze InceptionResNetV2
for layer in base_model.layers:
  layer.trainable = False

# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

In [ ]:
lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_resnet_freezed.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 1082
nb_validation_samples= 103
epochs=10
batch_size=16

history = model.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

In [ ]:
score = model.evaluate(validation_generator, verbose=1)

7/7 [==============================] - 2s 272ms/step - loss: 0.6589 - precision: 0.8709 - recall: 0.7411 - f1: 0.7970 - acc: 0.7767


- Loss : 0.6589
- Precision : 0.8709
- Recall : 0.7411
- f1_score : 0.7970
- accuracy : 0.7767

## Phase 2 : Retrains last layer of inception

In [ ]:
base_model.summary()

We can begin by unfreezing block8_9_mixed

In [5]:
base_model = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(334,334,3)
    )
for layer in base_model.layers[:len(base_model.layers)-26]:
  layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-26:]:
  layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_retrain = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_retrain.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

219062272/219055592 [==============================] - 2s 0us/step


In [ ]:
lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_resnet_retrained.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 1082
nb_validation_samples= 103
epochs=10
batch_size=16

history = model_retrain.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

In [8]:
model_retrain.evaluate_generator(train_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.2877050042152405,
 0.9331589341163635,
 0.8658088445663452,
 0.8972028493881226,
 0.9038817286491394]

In [9]:
score = model_retrain.evaluate(validation_generator, verbose=1)

7/7 [==============================] - 22s 3s/step - loss: 0.3850 - precision: 0.9109 - recall: 0.8482 - f1: 0.8777 - acc: 0.8932


- Loss : 0.3850
- Precision : 0.9109
- Recall : 0.8472
- f1_score : 0.8777
- accuracy : 0.8932

Well good news, unfrezing the last layers help us improve the performance of the model. Now let's try to unfreeze 15 more layers.

### Phase 3 : Retrain 15 more layers

In [ ]:
base_model = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(334,334,3)
    )
for layer in base_model.layers[:len(base_model.layers)-40]:
  layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-40:]:
  layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_retrain_v2 = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_retrain_v2.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

In [ ]:
lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_resnet_retrained_v2.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 1082
nb_validation_samples= 103
epochs=10
batch_size=16

history = model_retrain_v2.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

In [ ]:
score = model_retrain_v2.evaluate(validation_generator, verbose=1)

7/7 [==============================] - 1s 173ms/step - loss: 0.3974 - precision: 0.8748 - recall: 0.8099 - f1: 0.8401 - acc: 0.8350


- Loss : 0.3974
- Precision : 0.8748
- Recall : 0.8099
- f1_score : 0.8401
- accuracy : 0.8350

Well the performance are good but not as good as before

# Cropped Images

In [11]:
img_rows, img_cols = (224,224)
train_batchsize = 16
val_batchsize = 16

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      brightness_range=[0.2, 1.2],
      horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        orig_path + 'train_images_cropped/',
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical',
        interpolation='bicubic')
 
validation_generator = validation_datagen.flow_from_directory(
        orig_path + 'val_images_cropped/',
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False,
        interpolation='bicubic')

Found 941 images belonging to 20 classes.
Found 92 images belonging to 20 classes.


In [ ]:
base_model = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(224,224,3)
    )

# Freeze InceptionResNetV2
for layer in base_model.layers:
  layer.trainable = False

# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_resnet_cropped_freezed.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 941
nb_validation_samples= 92
epochs=10
batch_size=16

history = model.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=[lrate, checkpoint],
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)

In [ ]:
score = model.evaluate(validation_generator, verbose=1)

6/6 [==============================] - 1s 140ms/step - loss: 0.6957 - precision: 0.8516 - recall: 0.6771 - f1: 0.7487 - acc: 0.7609


- Loss : 0.6957
- Precision : 0.8516
- Recall : 0.6771
- f1_score : 0.7487
- accuracy : 0.7609

## Last Layer retrainable

In [12]:
base_model = InceptionResNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(224,224,3)
    )
for layer in base_model.layers[:len(base_model.layers)-26]:
  layer.trainable = False
for layer in base_model.layers[len(base_model.layers)-26:]:
  layer.trainable = True

# Add final layers
x = base_model.output
x = AveragePooling2D((4, 4), strides=(4, 4), name="avg_pool")(x)
x = Flatten(name="flatten")(x)
x = Dense(
          512,
          activation="swish",
          name="dense_1",
          kernel_initializer="he_uniform")(x)
x = Dropout(0.25)(x)
predictions = Dense(
    20,
    activation="softmax",
    name="predictions",
    kernel_initializer="he_uniform")(x)
model_retrain = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
adam = Adam(0.0001)
model_retrain.compile(loss="categorical_crossentropy",
              optimizer=adam,
             metrics=[precision, recall, f1, 'acc'])

lrate = LearningRateScheduler(step_decay)
checkpoint = ModelCheckpoint("/content/drive/My Drive/bird_classification/models/inception_resnet_cropped_retrained.h5",
                             monitor="val_acc",
                             mode="max",
                             save_best_only = True,
                             verbose=1)

nb_train_samples = 941
nb_validation_samples= 92
epochs=12
batch_size=16

#history = model_retrain.fit_generator(train_generator,
                                 #steps_per_epoch=nb_train_samples // batch_size,
                                 #epochs=epochs,
                                 #callbacks=[lrate, checkpoint],
                                 #validation_data=validation_generator,
                                 #validation_steps=nb_validation_samples // batch_size)

In [14]:
model_retrain.evaluate(train_generator)

59/59 [==============================] - 255s 4s/step - loss: 0.4832 - precision: 0.9030 - recall: 0.7615 - f1: 0.8239 - acc: 0.8427


[0.4832497835159302,
 0.9029626250267029,
 0.7614895701408386,
 0.8239474296569824,
 0.8427205085754395]

In [15]:
score = model_retrain.evaluate(validation_generator, verbose=1)

6/6 [==============================] - 20s 3s/step - loss: 0.4735 - precision: 0.8692 - recall: 0.8403 - f1: 0.8536 - acc: 0.8478
